# Data processing

Summary of the Task:

• Objective: Create a Jupyter notebook that accomplishes the following:

    o Utilize PySpark to read and transform the provided dataset.
    o Generate visualizations that other meaningful insights; be creative and showcase any interesting findings.
    o Include markdown explanations detailing your transformation methods and the rationale behind your visualizations.
• How to submit your workdone: Please upload the your notebook to GitHub and share us your github link

• Dataset Information: The data can be downloaded from Kaggle:
    o This dataset pertains to credit card transactions and requires data cleaning due to its messy nature.
    
• Key Considerations for Data Transformation:

    o Handling PII Data: Clearly explain your chosen methods for managing personally identifiable information (PII).
    o Data Quality Assurance: Describe how you identify and process dirty data.
    o JSON Flattening: Convert JSON data into a tabular format. The expected columns include:
```
    § Unnamed: 0
    § trans_date_trans_time (Transaction Time)
    § cc_num (Credit Card Number)
    § merchant (Merchant Name)
    § category (Merchant Category)
    § amt (Transaction Amount)
    § first (Credit Card Owner's First Name)
    § last (Credit Card Owner's Last Name)
    § gender (Credit Card Owner's Gender)
    § street (Credit Card Owner's Street Address)
    § city (Credit Card Owner's City)
    § state (Credit Card Owner's State)
    § zip (Credit Card Owner's Zip Code)
    § lat (Credit Card Owner's Latitude)
    § long (Credit Card Owner's Longitude)
    § city_pop (City Population)
    § job (Credit Card Owner's Job)
    § dob (Credit Card Owner's Date of Birth)
    § trans_num (Transaction Number)
    § merch_lat (Merchant Latitude)
    § merch_long (Merchant Longitude)
    § is_fraud (Fraud Case Indicator)
    § merch_zipcode (Merchant Zipcode)
    § merch_last_update_time (Merchant Last Update Time)
    § merch_eff_time (Merchant Effective Registration Time)
    § cc_bic (Credit Card BIC Code)
```
    o Timestamp Conversion: All time-related columns
(`trans_date_trans_time`, `merch_last_update_time`,
`merch_e?_time` columns) must be converted to a human-readable
timestamp format in UTC +8 timezone (e.g., YYYY-MM-DD HH:MM.SSSSSS Z).

o Name Derivation: Extract first and last name columns from
person_name:
§ The expected format is based on "first, last" but you may
encounter dirty data. Please process the name based on following
example:
§ person_name: "Edward, Sanchez" should result in First:
"Edward" and Last: "Sanchez".

o Candidates are expected to identify and clean any dirty data
independently.

In [1]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [2]:
import uuid
import pandas as pd
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, TimestampType

## Data processing using pyspark

Let us first set up spark session to do the necessary data cleansing

In [3]:
spark = SparkSession.builder \
    .appName("JSON Processing") \
    .master("spark://spark-master:7077") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .getOrCreate()

In [5]:
# read the data from the json file downloaded from kaggle

df = spark.read.json('data/cc_sample_transaction.json')

In [6]:
df.limit(5).show(truncate=True)

+----------+------+-------------+-----------+----------------+--------+----------------+----------------------+------------------+-----------+-------------+--------------------+--------------------+---------------------+--------------------+
|Unnamed: 0|   amt|     category|     cc_bic|          cc_num|is_fraud|  merch_eff_time|merch_last_update_time|         merch_lat| merch_long|merch_zipcode|            merchant|     personal_detail|trans_date_trans_time|           trans_num|
+----------+------+-------------+-----------+----------------+--------+----------------+----------------------+------------------+-----------+-------------+--------------------+--------------------+---------------------+--------------------+
|         0|  4.97|     misc_net|CITIUS33CHI|2703186189652095|       0|1325376018798532|         1325376018666|         36.011293| -82.048315|        28705|fraud_Rippin, Kub...|{"person_name":"J...|  2019-01-01 00:00:18|0b242abb623afc578...|
|         1|107.23|  grocery_pos

In [7]:
# Define the schema for the main JSON structure. All columns are in string type except for personal_detail. 
# This column is a nested json. We first define the schema for this column

main_schema = StructType([
    StructField("person_name", StringType()),
    StructField("gender", StringType()),
    StructField("address", StringType()), 
    StructField("lat", StringType()),
    StructField("long", StringType()),
    StructField("city_pop", StringType()),
    StructField("job", StringType()),
    StructField("dob", StringType())
])

# Define schema for the nested address
address_schema = StructType([
    StructField("street", StringType()),
    StructField("city", StringType()),
    StructField("state", StringType()),
    StructField("zip", StringType())
])

In [8]:
df_parsed = df.withColumn("personal_detail_parsed", F.from_json(F.col("personal_detail"), main_schema))

In [9]:
df_address = df_parsed.withColumn("address_parsed", F.from_json(F.col("personal_detail_parsed.address"), address_schema))

In [10]:
# Extract all fields
df_final = df_address.select(
    "*",
    F.col("personal_detail_parsed.person_name").alias("person_name"),
    F.col("personal_detail_parsed.gender").alias("gender"),
    F.col("address_parsed.street").alias("street"),
    F.col("address_parsed.city").alias("city"),
    F.col("address_parsed.state").alias("state"),
    F.col("address_parsed.zip").alias("zip"),
    F.col("personal_detail_parsed.lat").alias("latitude"),
    F.col("personal_detail_parsed.long").alias("longitude"),
    F.col("personal_detail_parsed.city_pop").alias("city_population"),
    F.col("personal_detail_parsed.job").alias("job"),
    F.col("personal_detail_parsed.dob").alias("dob")
).drop("personal_detail", "personal_detail_parsed", "address_parsed")

In [11]:
df_final.limit(1).show()

+----------+----+--------+-----------+----------------+--------+----------------+----------------------+---------+----------+-------------+--------------------+---------------------+--------------------+--------------------+------+--------------+--------------+-----+-----+--------+---------+---------------+--------------------+----------+
|Unnamed: 0| amt|category|     cc_bic|          cc_num|is_fraud|  merch_eff_time|merch_last_update_time|merch_lat|merch_long|merch_zipcode|            merchant|trans_date_trans_time|           trans_num|         person_name|gender|        street|          city|state|  zip|latitude|longitude|city_population|                 job|       dob|
+----------+----+--------+-----------+----------------+--------+----------------+----------------------+---------+----------+-------------+--------------------+---------------------+--------------------+--------------------+------+--------------+--------------+-----+-----+--------+---------+---------------+----------

In [12]:
# not working solution, since the timestamp columns are not of equal length. Suspected one column in millisecond, and one in nanosecond

dt_cols = ["merch_eff_time", "merch_last_update_time"]
for col_name in dt_cols:
    df_dt_converted = df_final.withColumn(col_name, F.date_format(
        F.from_utc_timestamp(
            (F.col(col_name) / 1000).cast(TimestampType()), 
            "Asia/Kuala_Lumpur"
        ),
        "yyyy-MM-dd HH:mm:ss"
    ))

In [13]:
df_dt_converted.select(["merch_eff_time", "merch_last_update_time"]).limit(2).show()

+----------------+----------------------+
|  merch_eff_time|merch_last_update_time|
+----------------+----------------------+
|1325376018798532|   2012-01-01 08:00:18|
|1325376044867960|   1974-03-15 07:30:04|
+----------------+----------------------+



In [14]:
# working solution, divided the timestamp accordingly

df_dt_converted = df_final.withColumn("merch_last_update_time", F.date_format(
        F.from_utc_timestamp(
            (F.col("merch_last_update_time") / 1000).cast(TimestampType()), 
            "Asia/Kuala_Lumpur"
        ),
        "yyyy-MM-dd HH:mm:ss"
    ))\
    .withColumn("merch_eff_time", F.date_format(
        F.from_utc_timestamp(
            (F.col("merch_eff_time") / 1000000).cast(TimestampType()), 
            "Asia/Kuala_Lumpur"
        ),
        "yyyy-MM-dd HH:mm:ss"
    ))\
    .withColumn("trans_date_trans_time", F.date_format(
        F.from_utc_timestamp(
            (F.col("trans_date_trans_time")).cast(TimestampType()), 
            "Asia/Kuala_Lumpur"
        ),
        "yyyy-MM-dd HH:mm:ss"
    ))

In [15]:
# the timestamp column looks good
df_dt_converted.select(["merch_eff_time", "merch_last_update_time", "trans_date_trans_time"]).limit(10).show(truncate=False)

+-------------------+----------------------+---------------------+
|merch_eff_time     |merch_last_update_time|trans_date_trans_time|
+-------------------+----------------------+---------------------+
|2012-01-01 08:00:18|2012-01-01 08:00:18   |2019-01-01 08:00:18  |
|2012-01-01 08:00:44|1974-03-15 07:30:04   |2019-01-01 08:00:44  |
|2012-01-01 08:00:51|2012-01-01 08:00:51   |2019-01-01 08:00:51  |
|2012-01-01 08:01:16|2012-01-01 08:01:16   |2019-01-01 08:01:16  |
|2012-01-01 08:03:06|1974-03-15 07:30:18   |2019-01-01 08:03:06  |
|2012-01-01 08:04:08|2012-01-01 08:04:08   |2019-01-01 08:04:08  |
|2012-01-01 08:04:42|2012-01-01 08:04:42   |2019-01-01 08:04:42  |
|2012-01-01 08:05:08|2012-01-01 08:05:08   |2019-01-01 08:05:08  |
|2012-01-01 08:05:18|2012-01-01 08:05:18   |2019-01-01 08:05:18  |
|2012-01-01 08:06:01|2012-01-01 08:06:01   |2019-01-01 08:06:01  |
+-------------------+----------------------+---------------------+



In [16]:
# the column person_name is dirty. we need to extract first and last name from this column

df_dt_converted.select(["person_name"]).limit(20).show(truncate=False)

+-------------------------+
|person_name              |
+-------------------------+
|Jennifer,Banks,eeeee     |
|Stephanie,Gill,eeeee     |
|Edward@Sanchez           |
|Jeremy/White, !          |
|Tyler@Garcia             |
|Jennifer,Conner,eeeee    |
|Kelsey, , Richards NOOOO |
|Steven, Williams         |
|Heather, , Chase NOOOO   |
|Melissa@Aguilar          |
|Eddie|Mendez!!!          |
|Theresa@Blackwell        |
|Charles|Robles!!!        |
|Jack@Hill                |
|Christopher@Castaneda    |
|Ronald@Carson            |
|Lisa, Mendez             |
|Nathan,Thomas,eeeee      |
|Justin, Gay              |
|Kenneth, , Robinson NOOOO|
+-------------------------+



In [17]:
# Normalize delimiters and remove unwanted characters, replace '@', '/', '|', and '!' with ','
df_normalized = df_dt_converted.withColumn(
    "person_name",
    F.regexp_replace("person_name", r"[@/|!]+", ",")  
)

# Split the column into an array
df_split = df_normalized.withColumn(
    "name_parts",
    F.split("person_name", ",")
)

# Extract first name, last name, and put the rest in other parts
df_final = df_split.withColumn("first_name", F.trim(df_split["name_parts"][0])) \
                   .withColumn("last_name", F.trim(df_split["name_parts"][1])) \
                   .withColumn("other_parts", F.trim(df_split["name_parts"][2]))

In [18]:
# some last name is in the other parts. extract it and put in last name column
df_final.select(["first_name", "last_name", "other_parts"]).limit(30).show(truncate=False)

+-----------+---------+--------------+
|first_name |last_name|other_parts   |
+-----------+---------+--------------+
|Jennifer   |Banks    |eeeee         |
|Stephanie  |Gill     |eeeee         |
|Edward     |Sanchez  |NULL          |
|Jeremy     |White    |              |
|Tyler      |Garcia   |NULL          |
|Jennifer   |Conner   |eeeee         |
|Kelsey     |         |Richards NOOOO|
|Steven     |Williams |NULL          |
|Heather    |         |Chase NOOOO   |
|Melissa    |Aguilar  |NULL          |
|Eddie      |Mendez   |              |
|Theresa    |Blackwell|NULL          |
|Charles    |Robles   |              |
|Jack       |Hill     |NULL          |
|Christopher|Castaneda|NULL          |
|Ronald     |Carson   |NULL          |
|Lisa       |Mendez   |NULL          |
|Nathan     |Thomas   |eeeee         |
|Justin     |Gay      |NULL          |
|Kenneth    |         |Robinson NOOOO|
+-----------+---------+--------------+
only showing top 20 rows



In [19]:
df_final = df_final.withColumn(
    "extracted_last_name",
    F.when(
        F.col("last_name").isNull() | (F.trim(F.col("last_name")) == ""),
        F.split(F.col("other_parts"), " ")[0]
    ).otherwise(F.col("last_name"))
)

df_final = df_final.drop("last_name", "other_parts", "name_parts").withColumnRenamed("extracted_last_name", "last_name")

In [20]:
# These two columns are now looking good
df_final.select(["first_name", "last_name"]).limit(30).show(truncate=False)

+-----------+---------+
|first_name |last_name|
+-----------+---------+
|Jennifer   |Banks    |
|Stephanie  |Gill     |
|Edward     |Sanchez  |
|Jeremy     |White    |
|Tyler      |Garcia   |
|Jennifer   |Conner   |
|Kelsey     |Richards |
|Steven     |Williams |
|Heather    |Chase    |
|Melissa    |Aguilar  |
|Eddie      |Mendez   |
|Theresa    |Blackwell|
|Charles    |Robles   |
|Jack       |Hill     |
|Christopher|Castaneda|
|Ronald     |Carson   |
|Lisa       |Mendez   |
|Nathan     |Thomas   |
|Justin     |Gay      |
|Kenneth    |Robinson |
+-----------+---------+
only showing top 20 rows



# Data analysis and visualization using pandas

Pyspark is not a equipped with the numerous tools to do data visualization. Let's convert this dataframe into pandas dataframe to further process and do data analysis

In [22]:
import matplotlib.pyplot as plt
import seaborn as sns

In [23]:
df_pd = df_final.toPandas()

In [24]:
# take a look at processed pandas dataframe
with pd.option_context('display.max_columns', None, 'display.max_rows', None, 'display.width', None):
    print(df_pd.head())

  Unnamed: 0     amt       category       cc_bic            cc_num is_fraud  \
0          0    4.97       misc_net  CITIUS33CHI  2703186189652095        0   
1          1  107.23    grocery_pos     ADMDUS41      630423337322        0   
2          2  220.11  entertainment         Null    38859492057661        0   
3          3    45.0  gas_transport  DEUTUS33TRF  3534093764340240        0   
4          4   41.96       misc_pos     APBCUS61   375534208663984        0   

        merch_eff_time merch_last_update_time           merch_lat  \
0  2012-01-01 08:00:18    2012-01-01 08:00:18           36.011293   
1  2012-01-01 08:00:44    1974-03-15 07:30:04  49.159046999999994   
2  2012-01-01 08:00:51    2012-01-01 08:00:51           43.150704   
3  2012-01-01 08:01:16    2012-01-01 08:01:16           47.034331   
4  2012-01-01 08:03:06    1974-03-15 07:30:18           38.674999   

    merch_long merch_zipcode                            merchant  \
0   -82.048315         28705          frau

In [25]:
# look at some basic info on the data
df_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296675 entries, 0 to 1296674
Data columns (total 27 columns):
 #   Column                  Non-Null Count    Dtype 
---  ------                  --------------    ----- 
 0   Unnamed: 0              1296675 non-null  object
 1   amt                     1296675 non-null  object
 2   category                1296675 non-null  object
 3   cc_bic                  1296675 non-null  object
 4   cc_num                  1296675 non-null  object
 5   is_fraud                1296675 non-null  object
 6   merch_eff_time          1296675 non-null  object
 7   merch_last_update_time  1296675 non-null  object
 8   merch_lat               1296675 non-null  object
 9   merch_long              1296675 non-null  object
 10  merch_zipcode           1100702 non-null  object
 11  merchant                1296675 non-null  object
 12  trans_date_trans_time   1296675 non-null  object
 13  trans_num               1296675 non-null  object
 14  person_name       

In [27]:
# save processed data to continue later
df_pd.to_parquet("data/processed_data.parquet")

In [26]:
# start from where we left off

df_pd = pd.read_parquet("data/processed_data.parquet")

In [27]:
# change to proper data type since the original data from the json file is mostly in string format

In [28]:
# list the column for easier copy and paste
df_pd.columns

Index(['Unnamed: 0', 'amt', 'category', 'cc_bic', 'cc_num', 'is_fraud',
       'merch_eff_time', 'merch_last_update_time', 'merch_lat', 'merch_long',
       'merch_zipcode', 'merchant', 'trans_date_trans_time', 'trans_num',
       'person_name', 'gender', 'street', 'city', 'state', 'zip', 'latitude',
       'longitude', 'city_population', 'job', 'dob', 'first_name',
       'last_name'],
      dtype='object')

In [29]:
# identify the columns that need to be converted to the correct data type
datetime_cols = ["merch_eff_time", "merch_last_update_time", "trans_date_trans_time", "dob"]
int_cols = ["city_population",]
float_cols = ["amt", "latitude", "longitude"]

In [30]:
df_pd[datetime_cols] = df_pd[datetime_cols].apply(lambda x: pd.to_datetime(x, errors='coerce'))
df_pd[int_cols] = df_pd[int_cols].astype("int64")
df_pd[float_cols] = df_pd[float_cols].astype("float64")

In [31]:
# now the columns are in right format
df_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296675 entries, 0 to 1296674
Data columns (total 27 columns):
 #   Column                  Non-Null Count    Dtype         
---  ------                  --------------    -----         
 0   Unnamed: 0              1296675 non-null  object        
 1   amt                     1296675 non-null  float64       
 2   category                1296675 non-null  object        
 3   cc_bic                  1296675 non-null  object        
 4   cc_num                  1296675 non-null  object        
 5   is_fraud                1296675 non-null  object        
 6   merch_eff_time          1296675 non-null  datetime64[ns]
 7   merch_last_update_time  1296675 non-null  datetime64[ns]
 8   merch_lat               1296675 non-null  object        
 9   merch_long              1296675 non-null  object        
 10  merch_zipcode           1100702 non-null  object        
 11  merchant                1296675 non-null  object        
 12  trans_date_tra

In [32]:
# peek at sample data
df_pd.head()

,Unnamed: 0,amt,category,cc_bic,cc_num,is_fraud,merch_eff_time,merch_last_update_time,merch_lat,merch_long,...,city,state,zip,latitude,longitude,city_population,job,dob,first_name,last_name
0,0,4.97,misc_net,CITIUS33CHI,2703186189652095,0,2012-01-01 08:00:18,2012-01-01 08:00:18,36.011293,-82.048315,...,Moravian Falls,NC,28654,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,Jennifer,Banks
1,1,107.23,grocery_pos,ADMDUS41,630423337322,0,2012-01-01 08:00:44,1974-03-15 07:30:04,49.159046999999994,-118.186462,...,Orient,WA,99160,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,Stephanie,Gill
2,2,220.11,entertainment,Null,38859492057661,0,2012-01-01 08:00:51,2012-01-01 08:00:51,43.150704,-112.154481,...,Malad City,ID,83252,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,Edward,Sanchez
3,3,45.00,gas_transport,DEUTUS33TRF,3534093764340240,0,2012-01-01 08:01:16,2012-01-01 08:01:16,47.034331,-112.561071,...,Boulder,MT,59632,46.2306,-112.1138,1939,Patent attorney,1967-01-12,Jeremy,White
4,4,41.96,misc_pos,APBCUS61,375534208663984,0,2012-01-01 08:03:06,1974-03-15 07:30:18,38.674999,-78.632459,...,Doe Hill,VA,24433,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,Tyler,Garcia


In [33]:
# look at basic statistic of the data
df_pd.describe()

,amt,merch_eff_time,merch_last_update_time,trans_date_trans_time,latitude,longitude,city_population,dob
count,1.296675e+06,1296675,1296675,1296675,1.296675e+06,1.296675e+06,1.296675e+06,1296675
mean,7.035104e+01,2008-11-18 09:24:34.822846464,2008-11-16 07:13:08.680719616,2019-10-03 20:47:28.070214400,3.853762e+01,-9.022634e+01,8.882444e+04,1973-10-03 19:02:55.017178512
min,1.000000e+00,1970-01-01 11:11:29,1970-06-03 17:07:24,2019-01-01 08:00:18,2.002710e+01,-1.656723e+02,2.300000e+01,1924-10-30 00:00:00
25%,9.650000e+00,2012-04-20 23:23:19.500000,2012-04-20 21:46:35,2019-06-04 03:12:22.500000,3.462050e+01,-9.679800e+01,7.430000e+02,1962-08-13 00:00:00
50%,4.752000e+01,2012-09-02 00:00:31,2012-09-01 23:44:46,2019-10-03 15:35:47,3.935430e+01,-8.747690e+01,2.456000e+03,1975-11-30 00:00:00
75%,8.314000e+01,2013-01-08 04:31:52.500000,2013-01-08 01:10:14,2020-01-28 23:02:55.500000,4.194040e+01,-8.015800e+01,2.032800e+04,1987-02-22 00:00:00
max,2.894890e+04,2013-06-21 20:13:37,2013-06-21 20:13:37,2020-06-21 20:13:37,6.669330e+01,-6.795030e+01,2.906700e+06,2005-01-29 00:00:00
std,1.603160e+02,NaN,NaN,NaN,5.075808e+00,1.375908e+01,3.019564e+05,NaN


In [34]:
df_pd.columns

Index(['Unnamed: 0', 'amt', 'category', 'cc_bic', 'cc_num', 'is_fraud',
       'merch_eff_time', 'merch_last_update_time', 'merch_lat', 'merch_long',
       'merch_zipcode', 'merchant', 'trans_date_trans_time', 'trans_num',
       'person_name', 'gender', 'street', 'city', 'state', 'zip', 'latitude',
       'longitude', 'city_population', 'job', 'dob', 'first_name',
       'last_name'],
      dtype='object')

## For PII purpose, lets split the table into customer and transaction

We will remove personal info from our transaction table when doing visualization

In [35]:
# list out all columns needed
user_cols = ['first_name', 'last_name', 'gender', 'street', 'city', 'state', 'zip', 'latitude', 'longitude', 'job', 'dob']

In [36]:
unique_user = df_pd.groupby(user_cols)[["amt"]].count().reset_index().drop("amt", axis=1)

In [37]:
# assign unique ID for each user, since we don't have one

unique_user['user_uuid'] = unique_user.apply(lambda _: str(uuid.uuid4()), axis=1)

In [38]:
unique_user.head()

,first_name,last_name,gender,street,city,state,zip,latitude,longitude,job,dob,user_uuid
0,Aaron,Murray,M,624 Hale Springs Apt. 572,Meadville,MO,64659,39.7795,-93.3014,Tourist information centre manager,1974-12-23,91370670-6c2d-4ab3-98a0-3ea6077cd33c
1,Aaron,Pena,M,793 Hooper Tunnel Suite 154,Burke,VA,22015,38.7894,-77.2818,Health visitor,1950-11-27,2ba307fb-669c-4747-bef5-d07618480a39
2,Aaron,Rogers,M,969 Huerta Path Apt. 270,Valentine,NE,69201,42.8062,-100.6215,Network engineer,1945-03-15,117eab10-665f-4522-8d2f-704eedcec564
3,Aaron,Stewart,M,400 Zuniga Pine,Winthrop,ME,4364,44.3229,-69.9576,Advertising account planner,1995-04-22,3bec9c77-15bf-40e1-bb93-631f5bfcc7ef
4,Adam,Keller,M,32600 Cobb Curve,Sardis,AL,36775,32.2844,-86.9920,Learning disability nurse,1932-09-17,632bca89-dc6e-4e09-b1e8-f2485b11c979


In [64]:
# save unique user to parquet for further analysis

unique_user.to_parquet("data/user.parquet")

In [39]:
# create credit card table per user 
df_pd_uid = df_pd.merge(unique_user[["first_name", "last_name", "dob", "user_uuid"]], on=["first_name", "last_name", "dob"], how="left")

In [40]:
df_pd_uid.groupby(['user_uuid', 'cc_num']).size().reset_index(name='count')

,user_uuid,cc_num,count
0,00135168-33cd-4ce9-9fb1-3beca2cf0756,375082648741747,2048
1,00969156-dc14-4cd8-a167-9f0087742e8e,571844099986,1542
2,00a7abce-66d3-4888-a8cc-933a1ad95dbf,180067784565096,2063
3,01750bd0-e82a-4900-9849-a6d71e5e13b1,213126662687660,1530
4,0197db6c-6151-49c2-8135-361e6b1dfe8d,4025612008285111,1514
...,...,...,...
978,feb2ef65-ee84-4444-9629-39d577d5a0fe,4334230547694630,2054
979,feb60bf3-a1f7-4264-ad5a-7b514d727133,371985236239474,1010
980,ff56d992-6a98-4b2e-897b-13c4b623ea8f,3501285649323933,9
981,ff985a5c-19eb-45ad-98dd-b3e2813e1be5,3559723917628303,1020


In [87]:
df_pd_uid.cc_num.str.len().sort_values()

397233     11
191384     11
509662     11
18668      11
1128272    11
           ..
1062416    19
360121     19
1062426    19
360155     19
1296674    19
Name: cc_num, Length: 1296675, dtype: int64

In [41]:
# create back transaction table without user information and only use UUID as identifier key

In [45]:
df_pd_uid.shape

(1296675, 28)

In [46]:
fact_table = df_pd_uid.drop(columns=user_cols).drop(columns='person_name')

In [47]:
# remove credit card info, we only take last 4 digits for reference

fact_table["cc_num"] = fact_table["cc_num"].str.strip().str[-4:]

In [48]:
fact_table.shape

(1296675, 16)

In [49]:
fact_table.head()

,Unnamed: 0,amt,category,cc_bic,cc_num,is_fraud,merch_eff_time,merch_last_update_time,merch_lat,merch_long,merch_zipcode,merchant,trans_date_trans_time,trans_num,city_population,user_uuid
0,0,4.97,misc_net,CITIUS33CHI,2095,0,2012-01-01 08:00:18,2012-01-01 08:00:18,36.011293,-82.048315,28705,"fraud_Rippin, Kub and Mann",2019-01-01 08:00:18,0b242abb623afc578575680df30655b9,3495,8df660dd-0f43-4f11-890d-41fba28e4633
1,1,107.23,grocery_pos,ADMDUS41,7322,0,2012-01-01 08:00:44,1974-03-15 07:30:04,49.159046999999994,-118.186462,None,"fraud_Heller, Gutmann and Zieme",2019-01-01 08:00:44,1f76529f8574734946361c461b024d99,149,5d969e3d-f198-4002-856b-7c55207b6168
2,2,220.11,entertainment,Null,7661,0,2012-01-01 08:00:51,2012-01-01 08:00:51,43.150704,-112.154481,83236,fraud_Lind-Buckridge,2019-01-01 08:00:51,a1a22d70485983eac12b5b88dad1cf95,4154,fe6551aa-83f8-40a1-a3fc-722e301d88e4
3,3,45.00,gas_transport,DEUTUS33TRF,0240,0,2012-01-01 08:01:16,2012-01-01 08:01:16,47.034331,-112.561071,None,"fraud_Kutch, Hermiston and Farrell",2019-01-01 08:01:16,6b849c168bdad6f867558c3793159a81,1939,0671017b-e583-4d5a-a282-0072df83723e
4,4,41.96,misc_pos,APBCUS61,3984,0,2012-01-01 08:03:06,1974-03-15 07:30:18,38.674999,-78.632459,22844,fraud_Keeling-Crist,2019-01-01 08:03:06,a41d7549acf90789359a9aa5346dcb46,99,06254c4b-34a1-42e1-81eb-1956e5b2ef6d


In [50]:
fact_table.to_parquet("data/fact.parquet")

# Data visualization

I've decided to use powerBI to do data analysis as I feel, it is the right tool for the job. All analysis result can be found in pdf file

![alt text](pg1.png)

- First page shows that the transaction happened in the US including Alaska and Hawaii.
- The transaction spread is pretty event with the following peaks
    1. 15/6/19
    2. 14/9/19
    3. 1/12/19
    4. 30/5/20

- grocery pos leads the category chart for transaction amount
- shopping_net on the other hands shows the highest percent of fraudulent cases

![image](pg2.png)

- image 2 clearly shows a different pattern between normal transaction cases and fraudulent cases
- fraudulent cases peak at around 6 and 7 in the morning (UTC+8) time
- normal transaction seems to be on peak from 8 pm to 7 am next day

![Alt Text](pg3.png)

User analysis

- There is quite even spread of the customers based on gender
- there is also some interesting careers. Some that catch my eyes
    - naval architect
    - exhibition designer
    - senior tax professional
- the user age group is also spreaded well from the earliest in 1924 to 2005. that's almost 80 year difference
- All users are also based in the US 

![Alt Text](pg4.png)

Top merchant (by amount) is totally different when filtered by fraudulent and non fraud transactions.